In [1]:
pip install folium

In [1]:
import pypsa
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import folium
import math
import networkx as nx

In [2]:
initial_network = pd.read_csv('database/lines_kalimantan.csv')

initial_network

,nodeA,nodeB,length_m
0,GI_Sambas,GI_Singkawang,59200
1,GI_Singkawang,GI_PLTU_2_Kalbar,20829
2,GI_PLTU_2_Kalbar,GI_PLTU_3_Kalbar,1393
3,GI_Singkawang,GI_Senggiring,70613
4,GI_Senggiring,GI_Parit_Baru,42848
...,...,...,...
57,GI_Bukuan,GI_Sambutan,7177
58,GI_Sambutan,GI_Muara_Badak,27134
59,GI_Muara_Badak,GI_Teluk_Pandan,57500
60,GI_Teluk_Pandan,GI_Sangatta,42347


In [3]:
initial_network_nodes = pd.read_csv('database/buses_substation_kalimantan.csv')

initial_network_nodes

,node,type,lon,lat,frequency,voltage
0,GI_Sambas,substation,109.2688,1.3429,50,150
1,GI_Singkawang,substation,108.9876,0.9266,50,150
2,GI_PLTU_3_Kalbar,substation,108.8747,0.8361,50,150
3,GI_PLTU_2_Kalbar,substation,108.8653,0.8300,50,150
4,GI_Senggiring,substation,108.9853,0.3162,50,150
...,...,...,...,...,...,...
57,GI_Muara_Badak,substation,117.2925,-0.3160,50,150
58,GI_Teluk_Pandan,substation,117.4350,0.1614,50,150
59,GI_Sangatta,substation,117.5693,0.4701,50,150
60,GI_Malinau,substation,116.6286,3.5114,50,150


In [4]:
combined = initial_network.merge(initial_network_nodes, left_on = "nodeA", right_on = "node", suffixes = ("_left", "_right"))
combined = combined.merge(initial_network_nodes, left_on  = "nodeB", right_on = "node", suffixes = ("_A", "_B"))
combined = combined [["nodeA", "lon_A", "lat_A", "nodeB", "lon_B", "lat_B"]]

display(combined)

,nodeA,lon_A,lat_A,nodeB,lon_B,lat_B
0,GI_Sambas,109.2688,1.3429,GI_Singkawang,108.9876,0.9266
1,GI_Bengkayang,109.4967,0.9056,GI_Singkawang,108.9876,0.9266
2,GI_Singkawang,108.9876,0.9266,GI_PLTU_2_Kalbar,108.8653,0.8300
3,GI_Singkawang,108.9876,0.9266,GI_Senggiring,108.9853,0.3162
4,GI_PLTU_2_Kalbar,108.8653,0.8300,GI_PLTU_3_Kalbar,108.8747,0.8361
...,...,...,...,...,...,...
57,GI_Bukuan,117.1930,-0.5832,GI_Sambutan,117.2139,-0.5242
58,GI_Sambutan,117.2139,-0.5242,GI_Muara_Badak,117.2925,-0.3160
59,GI_Muara_Badak,117.2925,-0.3160,GI_Teluk_Pandan,117.4350,0.1614
60,GI_Teluk_Pandan,117.4350,0.1614,GI_Sangatta,117.5693,0.4701


In [13]:
basemap = folium.Map(location = [-0.152375, 113.899132], zoom_start = 6.3)

maxvoltage = max(initial_network_nodes.voltage)

for i in range(len(initial_network_nodes)):
    popupinfo = (initial_network_nodes.node.iloc[i]+ ",voltage:" + str(initial_network_nodes.voltage.iloc[i]))
    folium.CircleMarker(location = [initial_network_nodes.lat.iloc[i], initial_network_nodes.lon.iloc[i]], 
                       tooltip = popupinfo, color = "green", fill = True, 
                       radius = math.sqrt(10 * initial_network_nodes.voltage.iloc[i] / maxvoltage)).add_to(basemap)

for i in range(len(combined)):
    points = [[combined.lat_A.iloc[i], combined.lon_A.iloc[i]],
             [combined.lat_B.iloc[i], combined.lon_B.iloc[i]]]
    folium.PolyLine(points, color = "yellow", opacitiy = 0.7).add_to(basemap)
    
basemap.save("initial_nodes.html")

display(basemap)